In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import platform
import platform
import random
from collections import (OrderedDict)
import jupyter_client
import jupyterlab
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly
import scipy
import seaborn as sns
from IPython.display import display
from notebook import __version__ as nb_ver

np.set_printoptions(precision=60, threshold=20, edgeitems=8, suppress=True, linewidth=999, sign=' ', floatmode='maxprec_equal')
SEED=1010336213
random.seed(SEED)
np.random.seed(SEED)


pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.precision', 3)
pd.set_option('display.html.border', 0)
pd.set_option('mode.chained_assignment', 'raise')
pd.set_option('display.colheader_justify', 'center')

plt.rcParams['agg.path.chunksize'] = 10000
plt.rcParams['figure.dpi'] = 100
plt.rcParams['figure.figsize'] = 16.0, 10.0
plt.rcParams['figure.titleweight'] = 'bold'
plt.rcParams['image.interpolation'] = 'None'
plt.rcParams['lines.linewidth'] = 1
plt.rcParams['savefig.bbox'] = 'tight'

sns.set()
sns.set(font_scale=0.8)
plt.style.use("dark_background")
sns.set_context(rc={'patch.linewidth': 0.0})
sns.set_style({
  'axes.axisbelow': False,
  'grid.color': '#ccc',
  'grid.alpha': 0.33,
  'grid.linestyle': '--',
  'grid.linewidth': 0.5,
  'axes.facecolor': '#111',
  'axes.edgecolor': '#ccc',
  'axes.grid': True,
  'axes.labelcolor': '#ccc',
  'figure.facecolor': '#111',
  'text.color': '#ccc',
  'xtick.color': '#ccc',
  'ytick.color': '#ccc',
  'patch.edgecolor': '#ccc',
})
sns.set_palette("pastel")

plt.rcParams['axes.grid'] = True
plt.rcParams['axes.grid.axis'] = 'both'
plt.rcParams['axes.grid.which'] = 'major'
plt.rcParams['grid.alpha'] = 0.33
plt.rcParams['grid.color'] = '#ccc'
plt.rcParams['grid.linestyle'] = '--'
plt.rcParams['grid.linewidth'] = 0.5

plt.rcParams['axes.spines.right'] = False
plt.rcParams['axes.spines.top'] = False

# Fixed random seed for better reproducibility
SEED = 4146516932
random.seed(SEED)
np.random.seed(SEED)

versions = pd.DataFrame.from_dict(OrderedDict([
  ('Operating system', ' '.join([platform.system(), platform.processor()])),
  ('Python', platform.python_version()),
  ('Jupyter', jupyter_client.__version__),
  ('Jupyter Notebook', nb_ver),
  ('Jupyter Lab', jupyterlab.__version__),
  ('Matplotlib', matplotlib.__version__),
  ('Seaborn', sns.__version__),
  ('Numpy', np.__version__),
  ('Pandas', pd.__version__),
  ('Plotly', plotly.__version__),
  ('SciPy', scipy.__version__),
]), orient='index')
versions.columns = ['Version']
display(versions)

,Version
Operating system,Linux x86_64
Python,3.12.1
Jupyter,8.6.1
Jupyter Notebook,7.1.2
Jupyter Lab,4.1.6
Matplotlib,3.8.2
Seaborn,0.13.2
Numpy,1.26.4
Pandas,2.2.0
Plotly,5.24.1


In [1]:
# import numpy
# numpy.show_config()

In [2]:
import ctypes
import sys
import glob
import os
import numpy as np

def check_library_backends():
    python_version = f"python{sys.version_info.major}.{sys.version_info.minor}"
    paths = [
        sys.exec_prefix + '/lib',
        os.path.join(os.path.expanduser('~/.local/lib'), python_version, 'site-packages', 'numpy.libs'),
    ]

    library_paths = []
    for path in paths:
        library_paths.extend(glob.glob(path + '/*blas*.so', recursive=True))
        library_paths.extend(glob.glob(path + '/*lapack*.so', recursive=True))

    if not library_paths:
        raise FileNotFoundError("No BLAS or LAPACK libraries found.")

    results = {}
    for lib_path in library_paths:
        lib = ctypes.CDLL(lib_path)
        backend = "Unknown"
        if hasattr(lib, 'openblas_get_config'):
            backend = "OpenBLAS"
        elif hasattr(lib, 'MKL_Get_Version_String'):
            backend = "MKL"
        elif hasattr(lib, 'ATL_buildinfo'):
            backend = "ATLAS"
        elif hasattr(lib, 'goto_get_num_procs'):
            backend = "GotoBLAS"
        elif hasattr(lib, 'blas_set_parameter'):
            backend = "BLIS"
        elif hasattr(lib, 'cblas_sgemm'):
            backend = "Apple Accelerate (BLAS)"
        if hasattr(lib, 'LAPACKE_dgetrf'):
            backend += " & Netlib LAPACK"
        elif hasattr(lib, 'dgels_'):
            backend += " & Apple Accelerate (LAPACK)"

        version = get_version(lib, backend) if backend != "Unknown" else "Version info not available"
        results[lib_path] = {"Backend": backend, "Version": version}

    return results

def get_version(lib, backend):
    version_info = "Version info not available"
    if backend.startswith("OpenBLAS"):
        get_config = lib.openblas_get_config
        get_config.restype = ctypes.c_char_p
        version_info = get_config().decode()
    elif backend.startswith("MKL"):
        mkl_get_version = lib.MKL_Get_Version_String
        mkl_get_version.restype = ctypes.c_char_p
        version_info = mkl_get_version().decode()
    return version_info

from pprint import pprint
pprint(check_library_backends())

{'/home/richard/micromamba/lib/libblas.so': {'Backend': 'OpenBLAS & Netlib '
                                                        'LAPACK',
                                             'Version': 'OpenBLAS 0.3.26 '
                                                        'DYNAMIC_ARCH '
                                                        'NO_AFFINITY Haswell '
                                                        'MAX_THREADS=128'},
 '/home/richard/micromamba/lib/libcblas.so': {'Backend': 'OpenBLAS & Netlib '
                                                         'LAPACK',
                                              'Version': 'OpenBLAS 0.3.26 '
                                                         'DYNAMIC_ARCH '
                                                         'NO_AFFINITY Haswell '
                                                         'MAX_THREADS=128'},
 '/home/richard/micromamba/lib/liblapack.so': {'Backend': 'OpenBLAS & Netlib '
                         

In [3]:
# Taken from treetime.py
def _eig_single_site(W, p):
    """
    Perform eigendecompositon of the rate matrix and stores the left- and right-
    matrices to convert the sequence profiles to the GTR matrix eigenspace
    and hence to speed-up the computations.
    NOTE: this assumes the diagonal of W is all zeros
    """
    assert np.abs(np.diag(W).sum())<1e-10

    tmpp = np.sqrt(p)
    symQ = W*np.outer(tmpp, tmpp)
    np.fill_diagonal(symQ, -np.sum(W*p, axis=1))

    eigvals, eigvecs = np.linalg.eigh(symQ)
    tmp_v = eigvecs.T*tmpp
    one_norm = np.sum(np.abs(tmp_v), axis=1)
    return eigvals, tmp_v.T/one_norm, (eigvecs*one_norm).T/tmpp

In [4]:
W = np.array([
  [0.00, 1.25, 1.25, 1.25, 1.25],
  [1.25, 0.00, 1.25, 1.25, 1.25],
  [1.25, 1.25, 0.00, 1.25, 1.25],
  [1.25, 1.25, 1.25, 0.00, 1.25],
  [1.25, 1.25, 1.25, 1.25, 0.00],
])

pi = np.array([0.2, 0.2, 0.2, 0.2, 0.2])

_eig_single_site(W, pi)

(array([-1.2500000000000002220446049250313, -1.2500000000000000000000000000000, -1.2500000000000000000000000000000, -1.2500000000000000000000000000000, -0.0000000000000002220446049250313]),
 array([[ 0.352941176470588092,  0.315789473684210564,  0.000000000000000000,  0.000000000000000000,  0.200000000000000011],
        [-0.441176470588235448,  0.184210526315789436, -0.070262648996307728,  0.150000000000000050,  0.200000000000000011],
        [-0.029411764705882353, -0.122807017543859670, -0.265791170012307509, -0.415153077165046569,  0.200000000000000011],
        [-0.029411764705882353, -0.122807017543859670,  0.499999999999999944, -0.084846922834953417,  0.200000000000000011],
        [ 0.147058823529411825, -0.254385964912280715, -0.163946180991384721,  0.349999999999999922,  0.200000000000000039]]),
 array([[ 1.03030303030303005, -1.28787878787878851, -0.08585858585858586, -0.08585858585858586,  0.42929292929292950],
        [ 1.38181818181818228,  0.80606060606060614, -0.5373737

In [5]:
W = np.array([
    [0.0, 1.0, 0.5, 0.3, 0.2],
    [1.0, 0.0, 0.8, 0.2, 0.4],
    [0.5, 0.8, 0.0, 0.7, 0.3],
    [0.3, 0.2, 0.7, 0.0, 0.6],
    [0.2, 0.4, 0.3, 0.6, 0.0]
])

pi = np.array([0.2, 0.2, 0.2, 0.2, 0.2])

_eig_single_site(W, pi)

(array([-0.684814487016582207346004906867165, -0.562331479193467154154006948374445, -0.430361364871948370858234511615592, -0.322492668918002101108299939369317,  0.000000000000000036441898489721763]),
 array([[ 0.202271628585528107,  0.307826383422367178, -0.098023838663632679,  0.250328116125804745, -0.200000000000000039],
        [-0.385290530500332129, -0.123608159555069985, -0.105959998204931419,  0.179928970165498470, -0.199999999999999956],
        [ 0.240360821331196267, -0.362050393692916472,  0.150219592451481615,  0.069742913708696827, -0.200000000000000094],
        [-0.114709469499667829,  0.192173616577632822,  0.349780407548518468, -0.177891061413691071, -0.199999999999999983],
        [ 0.057367550083275660, -0.014341446752013461, -0.296016163131435917, -0.322108938586308957, -0.199999999999999983]]),
 array([[ 0.76738605837507445, -1.46173036523939515,  0.91189033557986088, -0.43518929606296708,  0.21764326734742712],
        [ 1.10628265854551722, -0.44422950966770564, 

In [8]:
W = np.array([
    [0,  0.445,  0.64 ,  0.531,  0.425],
    [ 0.445, 0,  0.492,  0.571,  0.567],
    [ 0.64 ,  0.492, 0,  0.55 ,  0.459],
    [ 0.531,  0.571,  0.55 , 0,  0.435],
    [ 0.425,  0.567,  0.459,  0.435, 0],
])

pi = np.array([0.2, 0.2, 0.2, 0.2, 0.2])

ev, evec_left, evec_right = _eig_single_site(W, pi)

print("eigenvalues:\n", ev)

print("\nleft eigenvectors:\n", evec_left)

print("\nright eigenvectors:\n", evec_right)


eigenvalues:
 [-0.54897944125116571179034963279264 -0.53814016627092564615253422743990 -0.50166508082992933292842963055591 -0.45721531164797907242913765912817 -0.00000000000000013583064089189833]

left eigenvectors:
 [[-0.35360242252193302  0.09991699541026350  0.21776805850998729 -0.23687476891460404  0.20000000000000001]
 [-0.05193091765153580  0.35319333569605671 -0.26146358195171854  0.11282225035357402  0.20000000000000009]
 [ 0.50000000000000000  0.04688966889367960  0.09905586448759522 -0.15582104771802235  0.19999999999999996]
 [-0.07398730075763209 -0.37355261755853825 -0.23853641804828132 -0.10730418336737360  0.19999999999999996]
 [-0.02047935906889908 -0.12644738244146186  0.18317607700241764  0.38717774964642604  0.19999999999999996]]

right eigenvectors:
 [[-0.92173973465450487 -0.13536895453119385  1.30335608008643056 -0.19286359658328631 -0.05338379431744560]
 [ 0.34164553117988744  1.20767167074642279  0.16032953923657189 -1.27728603052363932 -0.43236071063924408]
 [ 1

In [9]:
evec_left.dot(evec_right)

array([[ 0.999999999999999666933092612453038,  0.000000000000000099920072216264091,  0.000000000000000160982338570647693,  0.000000000000000022204460492503126,  0.000000000000000099920072216264079],
       [ 0.000000000000000116573417585641417,  0.999999999999999111821580299874768,  0.000000000000000105471187339389829,  0.000000000000000105471187339389829,  0.000000000000000099920072216264054],
       [ 0.000000000000000116573417585641442,  0.000000000000000072164496600635165,  0.999999999999999333866185224906076,  0.000000000000000105471187339389891, -0.000000000000000066613381477509378],
       [ 0.000000000000000033306690738754701,  0.000000000000000210942374678779733,  0.000000000000000077715611723760978,  0.999999999999999222843882762390422,  0.000000000000000044408920985006276],
       [ 0.000000000000000088817841970012528,  0.000000000000000127675647831892992, -0.000000000000000088817841970012504,  0.000000000000000022204460492503151,  0.999999999999999888977697537484346]])

In [13]:
# useful test -- these should work for any matrix, also the degenerate ones
np.sum(np.abs(evec_left.dot(evec_right) - np.eye(5))) < 1e-12

True

In [14]:
# useful test
np.sum(np.abs(evec_left[:,-1] - pi)) < 1e-12

True

In [16]:
# useful test
np.sum(np.abs(evec_right[-1] - 1.0)) < 1e-12

True